In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

In [ ]:
df = pd.read_csv("../Dataset/gpreviews_2.csv")

In [4]:
df.head()

,reviewId,text,score,sentiment
0,7793317c-ab30-4e9b-abcd-6e3df0a5d440,Sony Liv. one of best experiences. I changed m...,1,Positive
1,c2617457-7693-45f0-9142-079b250ef22f,Horrible app honestly for such a rich company ...,0,Negative
2,dc96ac81-1311-4018-acee-588b79659125,This app very good and useful. For watching Ko...,1,Positive
3,5445cb50-f16d-4a5a-8b03-66909aa33bb8,Suddenly signed out of my account for no reaso...,0,Negative
4,8d86a93f-538d-451c-9243-f364227e3976,I've found everything here to be of my liking,1,Positive


In [5]:
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=device)
possible_labels = [
    "performance", "crashes", "features", "user experience",
    "cost", "ads", "version issues", "login problems", 
    "data loss", "language support", "customer support"
]
def get_labels(text):
    result = classifier(text, possible_labels, multi_label=True)
    return [label for label, score in zip(result["labels"], result["scores"]) if score > 0.6]

2025-04-12 10:13:47.449054: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744452827.629688      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744452827.682512      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
df["labels"] = df["text"].progress_apply(get_labels)

100%|██████████| 32998/32998 [1:59:17<00:00,  4.61it/s]


In [7]:
df.head()

,reviewId,text,score,sentiment,labels
0,7793317c-ab30-4e9b-abcd-6e3df0a5d440,Sony Liv. one of best experiences. I changed m...,1,Positive,"[user experience, features]"
1,c2617457-7693-45f0-9142-079b250ef22f,Horrible app honestly for such a rich company ...,0,Negative,"[user experience, cost, performance, features]"
2,dc96ac81-1311-4018-acee-588b79659125,This app very good and useful. For watching Ko...,1,Positive,"[features, user experience]"
3,5445cb50-f16d-4a5a-8b03-66909aa33bb8,Suddenly signed out of my account for no reaso...,0,Negative,"[login problems, data loss, user experience, c..."
4,8d86a93f-538d-451c-9243-f364227e3976,I've found everything here to be of my liking,1,Positive,"[features, user experience]"


In [ ]:
df.to_csv("../Dataset/gpreviews_2.csv", header = True, index = False)